In [11]:
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import openai
import tiktoken
import os
import warnings
warnings.filterwarnings('ignore')

# 환경변수 세팅
os.environ['OPENAI_API_KEY'] = '8c91c5c7058847f3820f9d7c12bbb1fb'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://sktfly-ai.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [6]:
loader = PyPDFLoader('data/K-패스 국토교통부 자료.pdf')
pages = loader.load_and_split()

tokenizer = tiktoken.get_encoding('cl100k_base')
tiktoken_len = lambda x:len(tokenizer.encode(x))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len,
)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [10]:
docs = text_splitter.split_documents(pages) # 문서를 적절한 크기로 나누어 준다.
db = Chroma.from_documents(docs, hf) # 문서를 임베딩하여 벡터스토어를 만든다.

In [12]:
chatgpt = AzureChatOpenAI(
    deployment_name='dev-gpt-35-turbo',
)

In [15]:
qa = RetrievalQA.from_chain_type(
    llm=chatgpt,
    chain_type='stuff',
    retriever=db.as_retriever(
        search_type='mmr',
        search_kwargs={'k':3, 'fetch_k':10} # fetch_k개의 문서를 가져온 후, 그 중에서 k개를 선택한다.
    ),
    return_source_documents=True # 답변에 사용된 문서를 함께 반환한다.
)

In [16]:
query = '청년에게 가장 큰 혜택을 주는 카드사는?'

result = qa(query)
result

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'query': '청년에게 가장 큰 혜택을 주는 카드사는?',
 'result': '알뜰교통카드가 청년에게 가장 큰 혜택을 주는 카드사입니다.',
 'source_documents': [Document(page_content='교통을 이용할 경우, 지출금액의 일정비율 일반인 청년층 ( 20%, 30%, \n저소득층 53%) 을 다음 달에 돌려받을 수 있는 교통카드로 현재 시행 , \n중인 알뜰교통카드의 이용 불편은 개선하고 혜택은 확대한 사업*이다.\n    * 기존 알뜰교통카드 도보ㆍ자전거 등 이동거리에 비례하여 마일리지 환급[ : ] 출도착 기록 필요( )\n개선 패스 이동거리 관계없이 지출금액의 일정 비율 환급[ : K- ] 출도착 기록 불필요( )\nㅇ 예를 들어 매월 대중교통비 로 평균 만원7을 지출하는 사람이라면, \n일반인은 만원 청년은 만원 저소득층은 만원 1.4 , 2.1 , 3.7 을 절감하여 \n연간 기준으로는 17 만원~44 까지 아낄 수 있으며 카드사의 추가 할인 , \n혜택최대 ( 10%) 까지 고려하면 교통비 절감 효과가 매우 크다 .\n□ 알뜰교통카드 를 사용하고 있는 이용자는 패스로의 회원 전환 절차K-*를 \n거치면 별도의 카드 재발급 없이 기존에 발급받은 교통카드를 그대로', metadata={'page': 0, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
  Document(page_content='패스K-를 사용한다면 전국 어디에서든 교통비 부담 을 획기적으로 덜 \n수 있을 것 이라며” ,\nㅇ 지자체 카드사 등 관계기관 협의 시스템 앱 개발 등 사업준비 절차를  “ · , ·\n빈틈없이 이행하여 월에 패스를 차질없이 출시하고 각 지자체와 5 K- , \n협력하여 패스를 기반으로 지역민 혜택은 더욱 강화할 수 있는 K- 지역\n맞춤형 패스 사업K- 도 적극 발굴 해나갈 계획 이라고 밝혔다 ” .\n담당 부서 대도시권광역교통위원회 책임자 과  장 백승록 (044-201-5080)